In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv("./../data/AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt", 
                   encoding='cp1252', 
                   sep="|", 
                   header=None,
                   na_filter=False)

In [3]:
data.columns = ["abbrev", "sense", "represntaion", "start_pos", "end_pos", "section_info", "sample"]

In [4]:
data.head(10)

,abbrev,sense,represntaion,start_pos,end_pos,section_info,sample
0,AB,abortion,AB.,231,233,,_%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida...
1,AB,abortion,AB.,249,251,,She is now bleeding quite heavily. Ultrasound ...
2,AB,abortion,AB,223,224,PAST OB HISTORY,ALLERGIES: Heparin and Imitrex. PAST OB HISTOR...
3,AB,abortion,AB.,194,196,HISTORY OF THE PRESENT ILLNESS,She had a pelvic ultrasound at Park Nicollet o...
4,AB,abortion,AB,114,115,PAST OB-GYN HISTORY,"On _%#MMDD2007#%_, normal anatomy with anterio..."
5,AB,ankle-brachial,AB,329,330,SIGNIFICANT FINDINGS,7. Laryngospasm. CONSULTANTS: 1. Nephrology. 2...
6,AB,abortion,AB,98,99,HISTORY OF PRESENT ILLNESS,HISTORY OF PRESENT ILLNESS: _%#NAME#%_ _%#NAME...
7,AB,blood group in ABO system,AB,292,293,PATIENT IDENTIFICATION,PATIENT IDENTIFICATION: _%#NAME#%_ _%#NAME#%_ ...
8,AB,abortion,AB,236,237,PAST MEDICAL HISTORY,PAST MEDICAL HISTORY: None except car accident...
9,AB,abortion,AB,65,66,,_%#NAME#%_ _%#NAME#%_ is a 25-year-old female ...


In [5]:
unique_abbrev = np.unique(data.abbrev)

In [6]:
empty_dict = dict.fromkeys(['abbrev','number_sense'])

In [7]:
abbrev_freq = pd.DataFrame(columns=["abbrev", "number_sense", "sense", "freq", "percentage"])
#abbrev_freq_dict = dict.fromkeys(['abbrev','number_sense'])

count_list = []
for abbrev in unique_abbrev:
    piece = data.loc[data.abbrev == abbrev]
    senses = np.unique(piece.sense)
    count = len(np.unique(piece.sense))
    for sense in senses:
        count_sense = piece.loc[piece.sense == sense].shape[0]
        percentage = count_sense/piece.shape[0]
        new = pd.DataFrame({"abbrev" : [abbrev],
                            "number_sense" : [count],
                            "sense" : [sense],
                            "freq" : [count_sense],
                            "percentage" : [percentage]})
        
        abbrev_freq = pd.concat([abbrev_freq, new])
    #count_list.append(count)
    
#abbrev_freq = pd.DataFrame({"abbrev" : unique_abbrev, "number_sense" : count_list})

In [28]:
np.unique(abbrev_freq.abbrev)

array(['AB', 'AC', 'ALD', 'AMA', 'ASA', 'AV', 'AVR', 'BAL', 'BK', 'BM',
       'BMP', 'C&S', 'C3', 'C4', 'CA', 'CDI', 'CEA', 'CR', 'CTA', 'CVA',
       'CVP', 'CVS', 'DC', 'DIP', 'DM', 'DT', 'EC', 'ER', 'ES', 'ET',
       'FISH', 'FSH', 'GT', 'IA', 'IB', 'IM', 'IR', 'IT', 'ITP', 'IVF',
       'LA', 'LE', 'MOM', 'MP', 'MR', 'MS', 'MSSA', 'NA', 'NAD', 'NP',
       'OP', 'OR', 'OTC', 'PA', 'PAC', 'PCP', 'PD', 'PDA', 'PE', 'PM',
       'PR', 'PT', 'RA', 'RT', 'SA', 'SBP', 'SMA', 'SS', 'T1', 'T2', 'T3',
       'T4', 'US', 'VAD', 'VBG'], dtype=object)

In [34]:
abbrev_freq.loc[abbrev_freq.abbrev == "AC"]

,abbrev,number_sense,sense,freq,percentage
0,AC,11,(drug) AC,161,0.322
0,AC,11,abdominal circumference,2,0.004
0,AC,11,acetate,4,0.008
0,AC,11,acromioclavicular,158,0.316
0,AC,11,adriamycin cyclophosphamide,118,0.236
0,AC,11,alternating current,1,0.002
0,AC,11,angiotensin-converting enzyme:ACE,3,0.006
0,AC,11,antecubital,1,0.002
0,AC,11,anticoagulation,1,0.002
0,AC,11,assist control,9,0.018


In [9]:
# feature extraction
# if select AC

In [10]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

In [52]:
def derive_features(abbrev, window_size, replace = True):  # replace = True, replace numbers with zero
    samples = data.loc[data.abbrev == abbrev, ]
    sample_num = 1
    
    original_features = pd.DataFrame(columns=["id", "features", "sense"])
    direction_features = pd.DataFrame(columns=["id", "features", "sense"])
    direction_num_features = pd.DataFrame(columns=["id", "features", "sense"])
    
    for i in range(samples.shape[0]):  # for each data point
        sentence_num = 1
        #target_word = samples.iloc[i, 2]  # should avoid using this, as word tokenization would split AC. to AC ., etc.
        #target_word = abbrev
        
        text = samples.iloc[i, 6]
        id = i+1  # this is the row number of selected abbreviation
        sense = samples.iloc[i, 1]
        
        target_word = samples.iloc[i, 2]
        sentence_length = 0
        start_pos = int(samples.iloc[i, 3])
        end_pos = int(samples.iloc[i, 4])
        
        target_word = text[start_pos : end_pos+1]
        detect = True
        #print(target_word)
        # sentence boundary
        # one sample can have multiple abbreviations in different sentences.
        # before sentence boundary, replace all the numbers with 0 
        #print(text)
        
        
        #if replace:
        #    text = re.sub("\d+", "0", text)
        #    text = re.sub("\d+\.\d+", "0", text)

        sents = sent_tokenize(text)
        exclude = set(string.punctuation)
        #s = ''.join(ch for ch in s if ch not in exclude)
        for sent in sents:  # for each sentence
            #words = [token.lower() for token in word_tokenize(sent)] # word tokenization
            # remove punctuations from the words list
            sentence_length += (len(sent)+1)
            words = [word for word in word_tokenize(sent) if word not in string.punctuation]
            
            print(id, "____________________")
            print(sent)
            print(words)
            print(target_word)
            print(sentence_length, end_pos)
            if sentence_length >= end_pos and (abbrev in words or target_word in words) and detect:
                #print(words)
                #print(sent)
                detect = False
                left_features = []
                right_features = []
                
                left_features_direction = []
                right_features_direction = []
                
                left_features_direction_num = []
                right_features_direction_num = []
                
                index = words.index(abbrev)
                # find the targeted word
                # 1. See if the window-size exceeds the front and back limit
                    # If yes, start from the zero-th element, towards right till find the target (features on the left)
                            # start from the max-th element, towards left till find the target (features on the right)
                    # If no, start from the (index - 5)-th element, towards right till find the target
                           # start from the (index - 5)-th element, towards left till find the target
                        
                # extract features on the left
                if index - window_size < 0:
                    j = 0
                    starting = index
                    #while words[j] != target_word:
                    while words[j] != abbrev:
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        # remove punctuation from the word, meant to fix problem in word tokenization
                        # but may cause problem, e.g. p.r.n --> prn
                        # may remove later
                        
                        left_features.append(words[j].lower())
                        left_features_direction.append("L-" + words[j].lower())
                        left_features_direction_num.append("L" + str(starting) + "-" + words[j].lower())
                        j += 1
                        starting -= 1
                else:
                    j = index-window_size
                    starting = 0
                    for k in range(window_size):
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        left_features.append(words[j].lower())
                        left_features_direction.append("L-" + words[j].lower())
                        left_features_direction_num.append("L" + str(window_size-starting) + "-" + words[j].lower())
                        j += 1
                        starting += 1
                
                # extract feature on the right
                if index + window_size >= len(words):
                    #j = len(words)-1
                    j = index+1
                    starting = 1
                    while j != len(words):
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        right_features.append(words[j].lower())
                        right_features_direction.append("R-" + words[j].lower())
                        right_features_direction_num.append("R" + str(starting) + "-" + words[j].lower())
                        j += 1
                        starting += 1
                else: 
                    j = index+1
                    starting = 1
                    for k in range(window_size):
                        words[j] = ''.join(ch for ch in words[j] if ch not in exclude)
                        right_features.append(words[j].lower())
                        right_features_direction.append("R-" + words[j].lower())
                        right_features_direction_num.append("R" + str(starting) + "-" + words[j].lower())
                        j += 1
                        starting += 1
                   
                left_features_str = " ".join(left_features)
                right_features_str = " ".join(right_features)
                features = left_features_str + " " + right_features_str
                
                print(features)
                
                left_features_direction_str = " ".join(left_features_direction)
                right_features_direction_str = " ".join(right_features_direction)
                features_direction = left_features_direction_str + " " + right_features_direction_str
                
                left_features_direction_num_str = " ".join(left_features_direction_num)
                right_features_direction_num_str = " ".join(right_features_direction_num)
                features_direction_num = left_features_direction_num_str + " " + right_features_direction_num_str
                #print(features_direction_num)
                
                if replace:
                    features = re.sub("\d+", "0", features)
                    features = re.sub("\d+\.\d+", "0", features)
                    
                    features_direction = re.sub("\d+", "0", features_direction)
                    features_direction = re.sub("\d+\.\d+", "0", features_direction)
                    
                    features_direction_num = re.sub("\d+", "0", features_direction_num)
                    features_direction_num = re.sub("\d+\.\d+", "0", features_direction_num)
                    
                original_features = pd.concat([original_features, pd.DataFrame({"id" : [id],
                                                                               "features" : [features], 
                                                                               "sense" : [sense]})])
                
                direction_features = pd.concat([direction_features, pd.DataFrame({"id" : [id],
                                                                               "features" : [features_direction], 
                                                                               "sense" : [sense]})])
                
                direction_num_features = pd.concat([direction_num_features, pd.DataFrame({"id" : [id],
                                                                               "features" : [features_direction_num], 
                                                                               "sense" : [sense]})])
            sentence_num += 1
        sample_num += 1
    return([original_features, direction_features, direction_num_features])
                

In [61]:
ar1, br1, cr1 = derive_features('ALD', 5, replace=True)

1 ____________________
She passed the ABR hearing screening test.
['She', 'passed', 'the', 'ABR', 'hearing', 'screening', 'test']
ALD.
43 241
1 ____________________
Immunizations: Hepatitis B vaccine was given on _%#MMDD2006#%_.
['Immunizations', 'Hepatitis', 'B', 'vaccine', 'was', 'given', 'on', 'MMDD2006']
ALD.
107 241
1 ____________________
_%#NAME#%_ was discharged on breastmilk or Enfamil 20 cal/oz taking 10-44 ml every 2-3 hours and/or breast-feeding every 2-3 hours ALD.
['NAME', 'was', 'discharged', 'on', 'breastmilk', 'or', 'Enfamil', '20', 'cal/oz', 'taking', '10-44', 'ml', 'every', '2-3', 'hours', 'and/or', 'breast-feeding', 'every', '2-3', 'hours', 'ALD']
ALD.
243 241
andor breastfeeding every 23 hours 
1 ____________________
The parents have scheduled an appointment to see you next Tuesday, _%#MMDD2007#%_.
['The', 'parents', 'have', 'scheduled', 'an', 'appointment', 'to', 'see', 'you', 'next', 'Tuesday', 'MMDD2007']
ALD.
326 241
1 ____________________
Thank you again for al

24 ____________________
A workup of UA/UC and ammonia was started to r/o UTI and metabolic diseases as causes of _%#NAME#%_'s excessive sleepiness that was sometimes difficult to arouse.
['A', 'workup', 'of', 'UA/UC', 'and', 'ammonia', 'was', 'started', 'to', 'r/o', 'UTI', 'and', 'metabolic', 'diseases', 'as', 'causes', 'of', 'NAME', "'s", 'excessive', 'sleepiness', 'that', 'was', 'sometimes', 'difficult', 'to', 'arouse']
ALD,
486 241
25 ____________________
She passed the ABR hearing screening test on _%#MMDD2002#%_.
['She', 'passed', 'the', 'ABR', 'hearing', 'screening', 'test', 'on', 'MMDD2002']
ALD.
61 134
25 ____________________
Immunizations: none given _%#NAME#%_ was discharged on breast-feeding ALD.
['Immunizations', 'none', 'given', 'NAME', 'was', 'discharged', 'on', 'breast-feeding', 'ALD']
ALD.
136 134
name was discharged on breastfeeding 
25 ____________________
The parents were asked to make an appointment to see on Monday, _%#MM#%_ _%#DD#%_, 2002.
['The', 'parents', 'were

47 ____________________
At the time of discharge (_%#MMDD2003#%_) _%#NAME#%_ was meeting all of his nutritional needs through a combination of breast and bottle feeding ALD every 2-3 hours.
['At', 'the', 'time', 'of', 'discharge', 'MMDD2003', 'NAME', 'was', 'meeting', 'all', 'of', 'his', 'nutritional', 'needs', 'through', 'a', 'combination', 'of', 'breast', 'and', 'bottle', 'feeding', 'ALD', 'every', '2-3', 'hours']
ALD
457 218
48 ____________________
Problems during the hospitalization included the following: 1.
['Problems', 'during', 'the', 'hospitalization', 'included', 'the', 'following', '1']
ALD
63 251
48 ____________________
FEN.
['FEN']
ALD
68 251
48 ____________________
Upon admission (_%#MMDD2003#%_) _%#NAME#%_ was started on D10 at 10cc/hr.
['Upon', 'admission', 'MMDD2003', 'NAME', 'was', 'started', 'on', 'D10', 'at', '10cc/hr']
ALD
142 251
48 ____________________
On _%#MMDD2003#%_, his IV was heplocked and he was allowed to breast-feed ALD with Enfamil supplementation ALD w

74 ____________________
The parents were asked to make an appointment for their child to see you within one week.
['The', 'parents', 'were', 'asked', 'to', 'make', 'an', 'appointment', 'for', 'their', 'child', 'to', 'see', 'you', 'within', 'one', 'week']
ALD.
291 199
74 ____________________
Follow-up clinic appointments scheduled at Fairview-University Children's Hospital include NICU follow-up clinic at 4 months corrected gestational age.
['Follow-up', 'clinic', 'appointments', 'scheduled', 'at', 'Fairview-University', 'Children', "'s", 'Hospital', 'include', 'NICU', 'follow-up', 'clinic', 'at', '4', 'months', 'corrected', 'gestational', 'age']
ALD.
443 199
75 ____________________
He passed the ABR hearing screening test.
['He', 'passed', 'the', 'ABR', 'hearing', 'screening', 'test']
ALD;
42 150
75 ____________________
Immunizations: Hepatitis B vaccine was given on _%#MMDD2004#%_.
['Immunizations', 'Hepatitis', 'B', 'vaccine', 'was', 'given', 'on', 'MMDD2004']
ALD;
106 150
75 _______

100 ____________________
The parents were asked to make an appointment for their child to see you within one week.
['The', 'parents', 'were', 'asked', 'to', 'make', 'an', 'appointment', 'for', 'their', 'child', 'to', 'see', 'you', 'within', 'one', 'week']
ALD.
286 194
100 ____________________
Thank you again for allowing us to share in the care of your patient.
['Thank', 'you', 'again', 'for', 'allowing', 'us', 'to', 'share', 'in', 'the', 'care', 'of', 'your', 'patient']
ALD.
356 194
101 ____________________
Physical exam was normal except for persistent jaundice in an awake, alert baby and a Grade II/VI systolic murmur.
['Physical', 'exam', 'was', 'normal', 'except', 'for', 'persistent', 'jaundice', 'in', 'an', 'awake', 'alert', 'baby', 'and', 'a', 'Grade', 'II/VI', 'systolic', 'murmur']
ALD.
115 173
101 ____________________
_%#NAME#%_ was discharged breast-feeding every 3 hours ALD.
['NAME', 'was', 'discharged', 'breast-feeding', 'every', '3', 'hours', 'ALD']
ALD.
175 173
discharged 

...

395 ____________________
The parents were asked to make an appointment for their child to see you within the week.
['The', 'parents', 'were', 'asked', 'to', 'make', 'an', 'appointment', 'for', 'their', 'child', 'to', 'see', 'you', 'within', 'the', 'week']
ALD.
207 115
396 ____________________
She passed the ABR hearing screening test.
['She', 'passed', 'the', 'ABR', 'hearing', 'screening', 'test']
ALD.
43 111
396 ____________________
Immunizations: _%#NAME#%_ was discharged on Neosure 22 taking it ALD.
['Immunizations', 'NAME', 'was', 'discharged', 'on', 'Neosure', '22', 'taking', 'it', 'ALD']
ALD.
113 111
on neosure 22 taking it 
396 ____________________
Mom is pumping breast milk at this time also.
['Mom', 'is', 'pumping', 'breast', 'milk', 'at', 'this', 'time', 'also']
ALD.
159 111
396 ____________________
The parents were asked to make an appointment for their child to see you Friday.
['The', 'parents', 'were', 'asked', 'to', 'make', 'an', 'appointment', 'for', 'their', 'child', 't

414 ____________________
The parents had and appointment to see you on _%#MMDD#%_ for a bilirubin check and on _%#MMDD#%_ for the newborn check.
['The', 'parents', 'had', 'and', 'appointment', 'to', 'see', 'you', 'on', 'MMDD', 'for', 'a', 'bilirubin', 'check', 'and', 'on', 'MMDD', 'for', 'the', 'newborn', 'check']
ALD.
395 273
415 ____________________
PKU/galactosemia/hypothyroidism/hemoglobinopathy/adrenal hyperplasia screens were sent and the results are normal at discharge.
['PKU/galactosemia/hypothyroidism/hemoglobinopathy/adrenal', 'hyperplasia', 'screens', 'were', 'sent', 'and', 'the', 'results', 'are', 'normal', 'at', 'discharge']
ALD,
128 246
415 ____________________
She passed the ABR hearing screening test.
['She', 'passed', 'the', 'ABR', 'hearing', 'screening', 'test']
ALD,
171 246
415 ____________________
_%#NAME#%_ was discharged on breast milk breast-feeding every 2-3 hours ALD, to be supplemented with pumped and bottled breast milk.
['NAME', 'was', 'discharged', 'on', 'b

439 ____________________
Several male cousins and uncles have had this diagnosis.
['Several', 'male', 'cousins', 'and', 'uncles', 'have', 'had', 'this', 'diagnosis']
ALD.
306 247
439 ____________________
The patient has had serial head MRIs with the last MRI on _%#MM#%_ 2005 showing worsening inactive demyelination which lead to a cord blood transplantation.
['The', 'patient', 'has', 'had', 'serial', 'head', 'MRIs', 'with', 'the', 'last', 'MRI', 'on', 'MM', '2005', 'showing', 'worsening', 'inactive', 'demyelination', 'which', 'lead', 'to', 'a', 'cord', 'blood', 'transplantation']
ALD.
463 247
440 ____________________
2.
['2']
ALD
3 203
440 ____________________
Hospitalized in _%#MM#%_ 2001 for pneumonia.
['Hospitalized', 'in', 'MM', '2001', 'for', 'pneumonia']
ALD
48 203
440 ____________________
PAST SURGICAL HISTORY: Central line placements.
['PAST', 'SURGICAL', 'HISTORY', 'Central', 'line', 'placements']
ALD
96 203
440 ____________________
FAMILY HISTORY: Mom with seasonal allergies.

464 ____________________
REVIEW OF SYSTEMS: No chest pain, palpitations, shortness of breath, pneumonia, nausea, or vomiting.
['REVIEW', 'OF', 'SYSTEMS', 'No', 'chest', 'pain', 'palpitations', 'shortness', 'of', 'breath', 'pneumonia', 'nausea', 'or', 'vomiting']
ALD.
364 261
465 ____________________
He had significant developmental delay, including loss of ability to understand speech and language as a result of ALD.
['He', 'had', 'significant', 'developmental', 'delay', 'including', 'loss', 'of', 'ability', 'to', 'understand', 'speech', 'and', 'language', 'as', 'a', 'result', 'of', 'ALD']
ALD.
120 118
language as a result of 
465 ____________________
He attended school through the fourth grade without problems prior to his diagnosis.
['He', 'attended', 'school', 'through', 'the', 'fourth', 'grade', 'without', 'problems', 'prior', 'to', 'his', 'diagnosis']
ALD.
205 118
465 ____________________
He underwent matched sibling donor bone marrow transplant on _%#MMDD2005#%_ with CMV/HSV nega

487 ____________________
SOCIAL HISTORY: He lives in Michigan with his mother, mother's roommate, maternal grandmother, and brother on a farm.
['SOCIAL', 'HISTORY', 'He', 'lives', 'in', 'Michigan', 'with', 'his', 'mother', 'mother', "'s", 'roommate', 'maternal', 'grandmother', 'and', 'brother', 'on', 'a', 'farm']
ALD.
507 195
488 ____________________
She passed the ABR hearing screening test.
['She', 'passed', 'the', 'ABR', 'hearing', 'screening', 'test']
ALD.
43 178
488 ____________________
Immunizations: None _%#NAME#%_ was discharged alternating breast-feeding with bottle EnfaCare 22kcal/ounce formula, every 2-3 hours ALD.
['Immunizations', 'None', 'NAME', 'was', 'discharged', 'alternating', 'breast-feeding', 'with', 'bottle', 'EnfaCare', '22kcal/ounce', 'formula', 'every', '2-3', 'hours', 'ALD']
ALD.
180 178
22kcalounce formula every 23 hours 
488 ____________________
The parents were asked to make an appointment for their child to see you during this coming week.
['The', 'parents'

In [62]:
set([i+1 for i in range(500)]) - set(ar1.id)

set()

In [59]:
ab = data.loc[data.abbrev == "AB"]

In [60]:
ab.iloc[483, 6]

'The patient was asked to be seen for an Internal Medicine consult per Dr. _%#NAME#%_ _%#NAME#%_. HISTORY OF PRESENT ILLNESS: Patient _%#NAME#%_. is a 46-year-old female admitted to station 10-North from Fairview Ridges ER. The parents state that the patient has been having increasing symptoms of anxiety, paranoia and agitation the past several weeks.'